# Homework 1

#### Nawat Ngerncham

In [1]:
import numpy as np
import numpy.random as rd
import pandas as pd

from numpy.random import default_rng
from dataclasses import dataclass
from typing import List, Callable, Tuple, TypeVar


@dataclass
class TSPResult:
    final_route: List[int]
    final_distance: float
    progress: pd.DataFrame


DEFAULT_N_ITERATIONS = 5_000_000

In [2]:
T = TypeVar("T")


def shuffle(lst: List[T]) -> List[T]:
    rng = default_rng()
    lst_cp = lst.copy()
    rng.shuffle(lst_cp)
    return lst_cp

## 2-OPT

In [3]:
def random_adj_idx(l: int) -> Tuple[int, int]:
    assert l >= 4
    pair1 = rd.randint(l)
    pair2 = rd.randint(l)

    while pair1 - 1 < pair2 < pair1 + 1:
        pair2 = rd.randint(l)

    return pair1, pair2


def two_opt_tsp(cities: List[int],
                path_distance: Callable[[List[int]], float],
                n_iters: int = DEFAULT_N_ITERATIONS) -> TSPResult:
    current_path = cities.copy()
    current_dist = path_distance(current_path)
    progress = [current_dist]

    for _ in range(n_iters):
        new_path = current_path.copy()

        pi1, pi2 = random_adj_idx(len(cities))
        new_path[pi1], new_path[pi2] = current_path[pi2], current_path[pi1]
        new_distance = path_distance(new_path)

        if new_distance < current_dist:
            current_path = new_path
            current_dist = new_distance

        progress.append(current_dist)

    return TSPResult(
        final_route=current_path,
        final_distance=current_dist,
        progress=pd.DataFrame({"distance": progress})
    )

## Random Sampling

In [4]:
def random_sampling_tsp(cities: List[int],
                        path_distance: Callable[[List[int]], float],
                        n_iters: int = DEFAULT_N_ITERATIONS) -> TSPResult:
    current_path = cities.copy()
    current_distance = path_distance(current_path)
    dist_progress = [current_distance]

    for _ in range(n_iters):
        new_path = shuffle(cities.copy())
        new_distance = path_distance(new_path)

        if new_distance < current_distance:
            current_distance = new_distance
            current_path = new_path

        dist_progress.append(current_distance)

    return TSPResult(
        final_route=current_path,
        final_distance=current_distance,
        progress=pd.DataFrame({"distance": dist_progress})
    )

## In-class 2-OPT Example

In [5]:
def in_class_2opt_distance(x: int, y: int):
    dist_matrix = [
        [0, 7, 7, 3, 8],
        [7, 0, 7, 5, 3],
        [7, 7, 0, 8, 5],
        [3, 5, 8, 0, 5],
        [8, 3, 5, 5, 0]
    ]

    return dist_matrix[x][y]


def in_class_path_distance(path: List[int]):
    path_to_use = path + [path[0]]
    return np.sum([in_class_2opt_distance(x, y) for x, y in zip(path_to_use[:-1], path_to_use[1:])])

In [6]:
# random_sampling_tsp([i for i in range(5)], in_class_path_distance, n_iters=250)

In [7]:
# two_opt_tsp([i for i in range(5)], in_class_path_distance, n_iters=250)

In [8]:
sgb128 = np.loadtxt("data/sgb128_dist.txt", dtype=np.int64)
sgb128

array([[   0,  966, 1513, ..., 1564, 2871,  348],
       [ 966,    0, 2410, ...,  198, 1917, 2541],
       [1513, 2410,    0, ..., 2217, 2673, 1570],
       ...,
       [1564,  198, 2217, ...,    0,  321,  936],
       [2871, 1917, 2673, ...,  321,    0,   34],
       [ 348, 2541, 1570, ...,  936,   34,    0]])

In [9]:
def sgb_path_dist(path: List[int]) -> float:
    return np.sum([sgb128[x, y] for x, y in zip(path[:-1], path[1:])])


cities = [i for i in range(128)]

In [10]:
rd_result = random_sampling_tsp(cities, sgb_path_dist)

TSPResult(final_route=[40, 43, 23, 77, 72, 49, 127, 15, 80, 70, 103, 87, 47, 62, 45, 90, 3, 71, 38, 108, 110, 42, 35, 44, 29, 118, 114, 13, 122, 28, 124, 24, 88, 9, 67, 81, 120, 11, 99, 101, 32, 64, 19, 54, 20, 56, 85, 107, 92, 116, 31, 33, 83, 94, 111, 1, 22, 26, 30, 57, 113, 14, 17, 97, 109, 25, 18, 125, 112, 65, 2, 58, 102, 41, 86, 37, 63, 79, 27, 84, 8, 4, 6, 95, 117, 16, 93, 0, 46, 121, 78, 100, 52, 105, 98, 126, 69, 12, 60, 75, 68, 73, 34, 123, 50, 36, 53, 96, 55, 89, 74, 66, 119, 61, 106, 48, 21, 91, 39, 51, 82, 5, 115, 76, 10, 59, 104, 7], final_distance=127904, progress=         distance
0          137322
1          137322
2          137322
3          137322
4          137322
...           ...
4999996    127904
4999997    127904
4999998    127904
4999999    127904
5000000    127904

[5000001 rows x 1 columns])

In [11]:
opt2_result = two_opt_tsp(cities, sgb_path_dist)

TSPResult(final_route=[32, 75, 2, 90, 4, 51, 54, 108, 88, 44, 42, 3, 10, 11, 27, 84, 53, 17, 69, 19, 110, 21, 72, 23, 79, 67, 15, 14, 29, 55, 63, 1, 31, 58, 34, 125, 106, 24, 116, 57, 0, 71, 36, 104, 33, 103, 35, 47, 16, 124, 113, 30, 9, 117, 12, 49, 97, 87, 68, 70, 122, 38, 77, 121, 6, 85, 66, 20, 56, 43, 73, 22, 101, 83, 76, 40, 59, 118, 112, 25, 81, 95, 62, 127, 65, 50, 8, 98, 7, 93, 74, 114, 39, 60, 61, 13, 105, 100, 28, 126, 89, 123, 37, 45, 18, 26, 64, 48, 92, 120, 52, 111, 99, 94, 115, 5, 119, 107, 102, 80, 78, 91, 41, 86, 46, 82, 109, 96], final_distance=40318, progress=         distance
0          137322
1          136851
2          136851
3          136851
4          136851
...           ...
4999996     40318
4999997     40318
4999998     40318
4999999     40318
5000000     40318

[5000001 rows x 1 columns])